# Import Data

In [1]:
import pandas as pd

In [2]:
nlp_dataset = pd.read_parquet('./arxiv_nlp.parquet', engine='pyarrow')

In [3]:
cc_dataset = pd.read_parquet('./arxiv_climate_change.parquet', engine='pyarrow')

# Preprocess Data

In [11]:
import nltk
nltk.download('stopwords')
import re
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import spacy

[nltk_data] Downloading package stopwords to /Users/moyi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Prepare stopwords
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
cc_abs = cc_dataset.abstract.values.tolist()
cc_ttl = cc_dataset.title.values.tolist()
nlp_abs = nlp_dataset.abstract.values.tolist()
nlp_ttl = nlp_dataset.title.values.tolist()

**try different bigrams model**

In [7]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            
    #deacc=True removes punctuations
docs_words = list(sent_to_words(cc_abs))

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

docs_words_nostops = remove_stopwords(docs_words)

Build bigram model using original text

In [ ]:
# Build the bigram model
bigram0 = gensim.models.Phrases(docs_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a bigram
bigram_mod0 = gensim.models.phrases.Phraser(bigram0)

Build bigram model using text without stopwords

In [ ]:
# Build the bigram model
bigram1 = gensim.models.Phrases(docs_words_nostops, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a bigram
bigram_mod1 = gensim.models.phrases.Phraser(bigram1)

In [ ]:
docs_words_bigrams0 = [bigram_mod0[doc] for doc in docs_words_nostops]

In [ ]:
docs_words_bigrams1 = [bigram_mod1[doc] for doc in docs_words_nostops]

In [ ]:
diff_list = []
diff_list1 = []
diff_list2 = []
for i in range(len(docs_words_bigrams0)):
    diff1 = [x for x in docs_words_bigrams0[i] if x not in docs_words_bigrams1[i]]
    diff2 = [x for x in docs_words_bigrams1[i] if x not in docs_words_bigrams0[i]]
    if len(diff1) != 0 or len(diff2) != 0:
        print(i)
    
    diff_list1 = diff_list1 + diff1
    diff_list2 = diff_list2 + diff2
    diff_list = diff_list + diff1 + diff2

In [ ]:
diff_list

In [ ]:
diff_list1

In [ ]:
diff_list2

In [ ]:
docs_words_bigrams0[0]

In [ ]:
docs_words_bigrams1[0]

## Clean texts

In [14]:
# functions

# Tokenize words and cleanup the text
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            
    #deacc=True removes punctuations

# Remove Stopwords, make bigrams and lemmatize
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def make_bigrams(texts):
    # Build the bigram model
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100) # higher threshold fewer phrases.

    # Faster way to get a sentence clubbed as a bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


def text_preprocess(docs): 
    # docs: list of str
      
    # Remove new line characters 
    docs = [re.sub('\s+', ' ', doc) for doc in docs]  
    # Remove distracting single quotes 
    docs = [re.sub("\'", "", doc) for doc in docs]

    docs_words = list(sent_to_words(docs))

    # Call preprocessing functions in order
    # Remove Stop Words
    docs_words_nostops = remove_stopwords(docs_words)

    # Form Bigrams
    docs_words_bigrams = make_bigrams(docs_words_nostops)

    # Do lemmatization keeping only noun, adj, vb, adv
    docs_lemmatized = lemmatization(docs_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Corpus 
    docs_cleaned = docs_lemmatized
    
    return docs_cleaned

In [27]:
cc_abs_words_cleaned = text_preprocess(cc_abs)
cc_abs_cleaned = list(map(lambda x: ' '.join(x), cc_abs_words_cleaned))

In [35]:
cc_ttl_words_cleaned = text_preprocess(cc_ttl)
cc_ttl_cleaned = list(map(lambda x: ' '.join(x), cc_ttl_words_cleaned))

In [36]:
nlp_abs_words_cleaned = text_preprocess(nlp_abs)
nlp_abs_cleaned = list(map(lambda x: ' '.join(x), nlp_abs_words_cleaned))

In [37]:
nlp_ttl_words_cleaned = text_preprocess(nlp_ttl)
nlp_ttl_cleaned = list(map(lambda x: ' '.join(x), nlp_ttl_words_cleaned))

## save preprocessed data

In [38]:
import json

In [39]:
save_data_dic = {"climate_change_abstract":cc_abs_cleaned, "climate_change_title":cc_ttl_cleaned, "nlp_abstract": nlp_abs_cleaned, "nlp_title": nlp_ttl_cleaned}


In [68]:
with open("preprocessed_data.json", 'w', encoding='utf-8') as fl:
    json.dump(save_data_dic, fl, ensure_ascii=False, indent=4)